### ECHO Metrics Functions and Code

##Total Harmonic Distortion: WORKING
Note there are some files where the fundamental frequency is too high for any type of THD calculation. Should ask Dr.Shah about what this means and how we can go about interpreting this for our benchmarking

In [ ]:
import numpy as np
import librosa
import scipy.signal

def calculate_thd_dynamic(audio_file):
    # Load the audio signal
    signal, sr = librosa.load(audio_file, sr=None)

    # Perform FFT on the signal
    fft_spectrum = np.fft.fft(signal)
    fft_magnitude = np.abs(fft_spectrum[:len(fft_spectrum) // 2])  # Only keep positive frequencies
    freqs = np.fft.fftfreq(len(signal), d=1/sr)[:len(fft_spectrum) // 2]

    # Find peaks in the FFT spectrum
    peaks, properties = scipy.signal.find_peaks(fft_magnitude, height=np.max(fft_magnitude) * 0.1)  # Consider peaks above 10% max amplitude

    if len(peaks) < 2:
        raise ValueError(f"Not enough peaks found to compute THD in file: {audio_file}")

    # Sort peaks by amplitude (descending)
    sorted_indices = np.argsort(properties["peak_heights"])[::-1]
    sorted_peaks = peaks[sorted_indices]

    # Assume the highest peak is the fundamental frequency
    fundamental_freq = freqs[sorted_peaks[0]]
    fundamental_amp = fft_magnitude[sorted_peaks[0]]

    # Compute THD by summing the power of the harmonics
    harmonic_power = 0
    for peak in sorted_peaks[1:]:  # Ignore fundamental, check harmonics
        harmonic_freq = freqs[peak]
        if np.isclose(harmonic_freq % fundamental_freq, 0, atol=1):  # Ensure it's a harmonic
            harmonic_power += fft_magnitude[peak] ** 2

    if harmonic_power == 0:
        raise ValueError(f"No harmonics found for THD calculation in file: {audio_file}")

    thd = (np.sqrt(harmonic_power) / fundamental_amp)
    return thd

In [ ]:
# prompt: run the thd funciton above on all .wav files

import numpy as np
import librosa
import glob

# ### ECHO Metrics Functions and Code
# ##Total Harmonic Distortion:
# Note there are some files where the fundamental frequency is too high for any type of THD calculation. Should ask Dr.Shah about what this means and how we can go about interpreting this for our benchmarking



# Find all .wav files in the current directory
wav_files = glob.glob("*.wav")

# Process each .wav file
for file in wav_files:
    thd = calculate_thd_dynamic(file)
    print(f"THD for {file}: {thd}")


##Noise Floor: (WORKING)
Note this is all calculated in decibels. Negative values indicate fractions of average signal power. For instance, -20 dB indicates that the signal's power is one-tenth of the reference power, while -40 dB means it's one-hundredth.

In [ ]:
import numpy as np
import librosa

def noiseFloor(audio_file, segment_duration=0.5):
    # Load the audio signal
    signal, sr = librosa.load(audio_file, sr=None)

    # Calculate segment length in samples
    segment_length = int(segment_duration * sr)

    # Split the signal into segments
    segments = [signal[i:i + segment_length] for i in range(0, len(signal), segment_length)]

    # Calculate RMS for each segment
    rms_values = [np.sqrt(np.mean(segment**2)) for segment in segments]

    # Average RMS value as the noise floor
    noise_floor = np.mean(rms_values)

    # Convert to decibels
    noise_floor_db = 20 * np.log10(noise_floor)

    return noise_floor_db

In [ ]:
# prompt: run noiseFloor(file) for every .wav file in this folder

import os

# Assuming the .wav files are in the current directory
for filename in os.listdir('.'):
  if filename.endswith('.wav'):
    noise_floor_value = noiseFloor(filename)
    print(f"Noise floor for {filename}: {noise_floor_value} dB")


Noise floor for D-N00-003.wav: -47.45856285095215 dB
Noise floor for P-N00-009.wav: -36.37200117111206 dB
Noise floor for N-M04-004.wav: -36.63763761520386 dB
Noise floor for I-F01-001.wav: -37.72143363952637 dB
Noise floor for H-N00-004.wav: -34.67665433883667 dB
Noise floor for O-N00-009.wav: -35.19062280654907 dB
Noise floor for C-N00-002.wav: -42.962965965270996 dB
Noise floor for M-M05-001.wav: -38.46359729766846 dB
Noise floor for B-M02-004.wav: -46.11800193786621 dB
Noise floor for F-M06-005.wav: -42.31052875518799 dB
Noise floor for G-N00-006.wav: -26.650385856628418 dB
Noise floor for J-F03-001.wav: -34.73475217819214 dB
Noise floor for E-M06-004.wav: -28.192243576049805 dB
Noise floor for K-N00-001.wav: -35.37731409072876 dB
Noise floor for A-M02-002.wav: -35.961761474609375 dB
Noise floor for L-N00-001.wav: -39.15335655212402 dB


##Dynamic Range (WORKING)


In [ ]:
### Insert Code Here

import numpy as np
import scipy.io.wavfile as wav

def calculate_dynamic_range(wav_file):
    """
    Calculates the dynamic range of a .wav file.

    Args:
        wav_file (str): Path to the .wav file.

    Returns:
        float: Dynamic range in decibels (dB).
    """
    # Read the wav file
    sample_rate, data = wav.read(wav_file)

    # Convert to mono if stereo
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)  # Take mean of channels to make mono

    # Compute peak amplitude
    peak_amplitude = np.max(np.abs(data))

    # Compute RMS amplitude
    rms_amplitude = np.sqrt(np.mean(data**2))

    # Avoid division by zero
    if rms_amplitude == 0:
        return float('inf')  # Infinite dynamic range if there is no signal

    # Compute dynamic range in dB
    dynamic_range = 20 * np.log10(peak_amplitude / rms_amplitude)

    return dynamic_range

# Example usage:
# print(calculate_dynamic_range("example.wav"))


In [ ]:
import os

# Assuming the .wav files are in the current directory
for filename in os.listdir('.'):
  if filename.endswith('.wav'):
    dyn = calculate_dynamic_range(filename)
    print(f"Dynamic range for {filename}: {dyn} dB")

##Crest Factor: (WORKING)

In [ ]:
### Ronoy

import numpy as np
import librosa

def crestFactor(audio_file, segment_duration=0.5):
    # Load the audio signal
    signal, sr = librosa.load(audio_file, sr=None)

    # Calculate segment length in samples
    segment_length = int(segment_duration * sr)

    # Split the signal into segments
    segments = [signal[i:i + segment_length] for i in range(0, len(signal), segment_length)]

    # Calculate peak and RMS for each segment
    crest_factors = []
    for segment in segments:
        peak = np.max(np.abs(segment))
        rms = np.sqrt(np.mean(segment**2))
        crest_factor = peak / rms if rms > 0 else 0  # Avoid division by zero
        crest_factors.append(crest_factor)

    # Average crest factor across segments
    average_crest_factor = np.mean(crest_factors)

    return average_crest_factor



In [ ]:
# prompt: run crest factor for every .wav file in this folder

import numpy as np
import librosa
import os

# ... (rest of your code, including the noiseFloor and calculate_thd_dynamic functions)

# Assuming the .wav files are in the current directory
for filename in os.listdir('.'):
  if filename.endswith('.wav'):
    noise_floor_value = noiseFloor(filename)
    print(f"Noise floor for {filename}: {noise_floor_value} dB")

    crest_factor_value = crestFactor(filename)
    print(f"Crest factor for {filename}: {crest_factor_value}")





```
```

##Waveform Complexity Index:

In [ ]:
import numpy as np
import librosa
import scipy.stats

def calculate_waveform_complexity(audio_file):
    # Load audio file
    signal, sr = librosa.load(audio_file, sr=None, mono=True)

    # Compute Zero-Crossing Rate (ZCR) - Measures frequency of sign changes
    zcr = librosa.feature.zero_crossing_rate(y=signal)
    avg_zcr = np.mean(zcr)

    # Compute Spectral Entropy manually (Energy Distribution across Frequencies)
    spectrum = np.abs(librosa.stft(signal))  # Compute magnitude spectrum
    spectrum = spectrum / np.sum(spectrum, axis=0, keepdims=True)  # Normalize
    spectral_entropy = scipy.stats.entropy(spectrum, axis=0)  # Compute entropy
    avg_entropy = np.mean(spectral_entropy)

    # Compute RMS Energy Variation - Measures fluctuations in loudness
    rms_energy = librosa.feature.rms(y=signal)
    rms_variation = np.std(rms_energy)  # Standard deviation to measure fluctuation

    # Compute Waveform Complexity Index (WCI)
    wci = (avg_zcr * avg_entropy) / (1 + rms_variation)  # Normalize by dynamic range

    print(f"Waveform Complexity Index for {audio_file}: {wci:.4f}")
    return wci



In [ ]:
# prompt: run waveform complexity (WCI)  factor for every .wav file in this folder

import numpy as np
import librosa
import scipy.signal
import glob
import os
import scipy.io.wavfile as wav



# Find all .wav files in the current directory
wav_files = glob.glob("*.wav")

# Process each .wav file
for file in wav_files:
    calculate_waveform_complexity(file)


Waveform Complexity Index for D-N00-003.wav: 0.3343
Waveform Complexity Index for P-N00-009.wav: 0.0594
Waveform Complexity Index for N-M04-004.wav: 0.1023
Waveform Complexity Index for I-F01-001.wav: 0.1686
Waveform Complexity Index for H-N00-004.wav: 0.4746
Waveform Complexity Index for O-N00-009.wav: 0.1211
Waveform Complexity Index for C-N00-002.wav: 0.3496
Waveform Complexity Index for M-M05-001.wav: 0.1596
Waveform Complexity Index for B-M02-004.wav: 0.5198
Waveform Complexity Index for F-M06-005.wav: 0.2073
Waveform Complexity Index for G-N00-006.wav: 0.5155
Waveform Complexity Index for J-F03-001.wav: 0.3211
Waveform Complexity Index for E-M06-004.wav: 0.0819
Waveform Complexity Index for K-N00-001.wav: 0.1762
Waveform Complexity Index for A-M02-002.wav: 0.4618
Waveform Complexity Index for L-N00-001.wav: 0.1828


##SNR: WORKING

In [ ]:
### Ronoy

import numpy as np
import librosa

def snr(audio_file, noise_duration=0.5, signal_duration=2.0):
    # Load the audio signal
    signal, sr = librosa.load(audio_file, sr=None)

    # Calculate noise and signal lengths in samples
    noise_length = int(noise_duration * sr)
    signal_length = int(signal_duration * sr)

    # Get noise and signal segments
    noise_segment = signal[:noise_length]
    signal_segment = signal[noise_length:noise_length + signal_length]

    # Calculate RMS of noise and signal
    noise_rms = np.sqrt(np.mean(noise_segment**2))
    signal_rms = np.sqrt(np.mean(signal_segment**2))

    # Calculate SNR in decibels
    snr_db = 20 * np.log10(signal_rms / noise_rms) if noise_rms > 0 else float('inf')  # Avoid division by zero

    return snr_db



In [ ]:
# prompt: run snr on all .wav files

import numpy as np
import librosa
import os

# ... (Your existing code for THD, noiseFloor, crestFactor, and snr functions)

# Assuming the .wav files are in the current directory
for filename in os.listdir('.'):
    if filename.endswith('.wav'):
        try:
            snr_value = snr(filename)
            print(f"SNR for {filename}: {snr_value} dB")

        except Exception as e:
            print(f"Error processing {filename}: {e}")


In [ ]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp311-cp311-linux_x86_64.whl size=542097 sha256=1557271c63891b150f8594fc9b42139dd5b1fa30b6974f3a498385a410dbfb64
  Stored in directory: /root/.cache/pip/wheels/5c/8a/f6/fd3df9a9714677410a5ccbf3ca519e66db4a54a1c46ea95332
Successfully built fastdtw


In [ ]:
### Lily: This is SNR with alignment preprocessing.

import wave
import numpy as np
from scipy.signal import spectrogram
import librosa
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import soundfile as sf
import matplotlib.pyplot as plt

# Redefining file paths
file1_path = 'F-F03-010.WAV'
file2_path = 'E-F01-006.WAV'

# Function to read WAV file and extract audio data
def read_wav(file_path):
    try:
        with wave.open(file_path, 'r') as wav_file:
            # Extract properties
            n_channels = wav_file.getnchannels()
            sample_width = wav_file.getsampwidth()
            framerate = wav_file.getframerate()
            n_frames = wav_file.getnframes()

            # Read frames and convert to numpy array
            audio_frames = wav_file.readframes(n_frames)
            audio_data = np.frombuffer(audio_frames, dtype=np.int16)

            # Handle stereo audio by taking only one channel if necessary
            if n_channels > 1:
                audio_data = audio_data[::n_channels]

            return audio_data, framerate
    except Exception as e:
        return None, None

# Read both audio files
audio1, sr1 = read_wav(file1_path)
audio2, sr2 = read_wav(file2_path)

# Check the basic properties of the audio data
audio1_properties = (len(audio1), sr1) if audio1 is not None else None
audio2_properties = (len(audio2), sr2) if audio2 is not None else None

print(audio1_properties, audio2_properties)

# Function to calculate the spectrogram for alignment purposes
def calculate_spectrogram(audio, sr, nperseg=512, noverlap=256):
    f, t, Sxx = spectrogram(audio, fs=sr, nperseg=nperseg, noverlap=noverlap)
    return f, t, Sxx

# Calculate spectrograms for both audio files
f1, t1, Sxx1 = calculate_spectrogram(audio1, sr1)
f2, t2, Sxx2 = calculate_spectrogram(audio2, sr2)

# Check the spectrogram dimensions for alignment feasibility
Sxx1_shape = Sxx1.shape
Sxx2_shape = Sxx2.shape

print(Sxx1_shape, Sxx2_shape)

# Function to load audio and compute MFCCs
def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return y, sr, mfcc

# Load your audio files
file1_path = "F-F03-010.WAV"
file2_path = "E-F01-006.WAV"
y1, sr1, mfcc1 = extract_mfcc(file1_path)
y2, sr2, mfcc2 = extract_mfcc(file2_path)

# Perform DTW on flattened MFCCs
distance, path = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)

print(f"DTW Distance: {distance}")
print(f"Path Length: {len(path)}")

def get_aligned_segments(path, frame_length, sr):
    aligned_segments = []
    current_segment = []
    last_frame1, last_frame2 = -1, -1

    for frame1, frame2 in path:
        if last_frame1 + 1 == frame1 and last_frame2 + 1 == frame2:
            # Continue current segment
            current_segment.append((frame1, frame2))
        else:
            # Save completed segment
            if current_segment:
                aligned_segments.append(current_segment)
            current_segment = [(frame1, frame2)]  # Start a new segment

        last_frame1, last_frame2 = frame1, frame2

    # Append the final segment
    if current_segment:
        aligned_segments.append(current_segment)

    # Convert frame indices to time intervals
    segments_in_time = [
        ((seg[0][0] * frame_length) / sr, (seg[-1][0] * frame_length) / sr)
        for seg in aligned_segments
    ]
    return segments_in_time

frame_length = 512  # Window size used in the spectrogram
aligned_time_segments = get_aligned_segments(path, frame_length, sr1)
print(aligned_time_segments)

def extract_segments(audio, sr, time_segments):
    segments = []
    for start_time, end_time in time_segments:
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        segments.append(audio[start_sample:end_sample])
    return segments

# Extract speech segments
speech_segments = extract_segments(audio1, sr1, aligned_time_segments)

def extract_noise_segments(audio, sr, time_segments):
    noise_segments = []
    last_end = 0
    for start_time, end_time in time_segments:
        start_sample = int(start_time * sr)
        if start_sample > last_end:
            noise_segments.append(audio[last_end:start_sample])
        last_end = int(end_time * sr)
    if last_end < len(audio):
        noise_segments.append(audio[last_end:])
    return noise_segments

# Extract noise segments
noise_segments = extract_noise_segments(audio1, sr1, aligned_time_segments)

# Save a speech segment
sf.write("speech_segment.wav", speech_segments[0], sr1)

# Save a noise segment
sf.write("noise_segment.wav", noise_segments[0], sr1)


plt.figure(figsize=(10, 4))
plt.plot(audio1, label="Original Audio")
for start_time, end_time in aligned_time_segments:
    plt.axvspan(start_time * sr1, end_time * sr1, color="green", alpha=0.3, label="Speech")
plt.legend()
plt.title("Speech Segments")
plt.show()

def calculate_snr(signal, noise):
    signal_energy = np.sum(signal ** 2)
    noise_energy = np.sum(noise ** 2)
    snr = 10 * np.log10(signal_energy / noise_energy)
    return snr

# Example usage (replace with actual segmented signals and noise):
speech_signal = y1[:10000]  # Example segment for speech
background_noise = y1[10000:20000]  # Example segment for noise
snr_value = calculate_snr(speech_signal, background_noise)
print(f"SNR: {snr_value:.2f} dB")

speech_segments = extract_segments(audio2, sr2, aligned_time_segments)
noise_segments = extract_noise_segments(audio2, sr2, aligned_time_segments)
sf.write("speech_segment.wav", speech_segments[0], sr2)
speech_signal = y2[:10000]  # Example speech segment from audio2
background_noise = y2[10000:20000]  # Example noise segment from audio2
snr_value = calculate_snr(speech_signal, background_noise)
print(f"SNR: {snr_value:.2f} dB")

None None


AttributeError: 'NoneType' object has no attribute 'shape'